<a href="https://colab.research.google.com/github/Muhammad-Yunus/Colab/blob/main/YOLOV5_training_for_Vision_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will guide you to train your own AI model using YOLOv5!

**Step 1.** Choose **GPU** in **Runtime** if not already selected by navigating to `Runtime --> Change Runtime Type --> Hardware accelerator --> GPU`

**Step 2.** Clone repo, install dependencies and check PyTorch and GPU

In [ ]:
!git clone https://github.com/Seeed-Studio/yolov5-swift  # clone
%cd yolov5-swift


In [ ]:
%pip install -qr requirements.txt  # install dependencies

import torch
import os
from google.colab import files
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

**Step 3.** Set up environment

In [ ]:
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

**Step 4.** Copy and paste the displayed code snippet from Roboflow on to the code cell below

<div align=center><img width=500 src="https://files.seeedstudio.com/wiki/YOLOV5/81.png"/></div>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="uwcaZm715IIKM3Ej7ix5")
print(rf.workspace())
project = rf.workspace("learning-k0ow4").project("lego-classification-brvrs")
version = project.version(1)
dataset = version.download("yolov5")


In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

**Step 5.** Download a pre-trained model suitable for our training

In [ ]:
!wget https://github.com/Seeed-Studio/yolov5-swift/releases/download/v0.1.0-alpha/yolov5n6-xiao.pt

**Step 6.** Start training

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [ ]:
!python3 train.py --img 192 --batch 64 --epochs 100 --data {dataset.location}/data.yaml --cfg yolov5n6-xiao.yaml --weights yolov5n6-xiao.pt --name yolov5n6_results --cache

**Step 7.** Export TensorFlow Lite file

In [ ]:
!python3 export.py --data {dataset.location}/data.yaml --weights runs/train/yolov5n6_results/weights/best.pt --imgsz 192 --int8 --include tflite

**Step 8.** Convert TensorFlow Lite to UF2 file

UF2 is a file format, developed by Microsoft. Seeed uses this format to convert .tflite to .uf2, allowing tflite files to be stored on the AIoT devices launched by Seeed. Currently Seeed's devices support up to 4 models, each model (.tflite) is less than 1M .

You can specify the model to be placed in the corresponding index with -t.

For example:

- `-t 1`: index 1
- `-t 2`: index 2

In [ ]:
# Place the model to index 1
!python3 uf2conv.py -f GROVEAI -t 1 -c runs//train/yolov5n6_results//weights/best-int8.tflite -o model-1.uf2
%cp model-1.uf2 ../

**Step 9.** Download the trained model file

In [ ]:
files.download("/content/model-1.uf2")

The above is the file that we will load into the SenseCAP A1101/ Grove - Vision AI Module to perform the inference!